In [26]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import sequence

import tensorflow as tf

from tensorflow.keras import layers

from keras.utils.vis_utils import plot_model

from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input, Dropout, Embedding
from keras.utils.vis_utils import plot_model

# Provides function for sliding window for training data of time series data https://www.tensorflow.org/tutorials/structured_data/time_series#the_weather_dataset
# How do we use our LSTM model as an input to a dense model with member features?
# Reference for classification with LSTM https://keras.io/examples/imdb_lstm/
# TODO Multi-input

# Functional API Example

In [192]:
# Next number prediction!
# Creates a random incremental sequence of numbers from 0 to some integer
# Each observation has variable length, but padding is applied so observations start with 0's

batch_size = 100
num_batch = 250
num_integers = 10
max_integer = num_integers - 1
epochs = 3
lstm_units = 16
embedding_size = 16

# Generate Sequential input data of variable timestep length
# Example: [0, 1, 2, 3, 4]
x_train = []
y_train = []
for i in range(num_batch*batch_size):
    timestep_len = np.random.randint(max_integer)+1
    x_train.append(np.arange(0, timestep_len+1))
    y_obs = np.zeros(num_integers)
    y_obs[timestep_len] = 1
    y_train.append(y_obs)
x_train = np.array(x_train)

# Pad input so all observations are of same length
# Example: [0, 0, 0, 0, 0, 0, 1, 2, 3, 4]
x_train = sequence.pad_sequences(x_train, dtype='int32', maxlen = num_integers)
y_train = np.array(y_train)

data_in = Input(shape = (num_integers,))
x = Embedding(num_integers, embedding_size)(data_in)
#x = LSTM(lstm_units, return_sequences = True)(x) - stacked LSTM layers doesn't seem to help in this case
x = LSTM(lstm_units)(x)
x = Dropout(0.2)(x)
output = Dense(num_integers, activation='softmax')(x)

model = Model(data_in, output)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

#model.summary()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1)


Epoch 1/3
25000/25000 [==============================] - 10s 417us/step - loss: 1.3839 - acc: 0.5557
Epoch 2/3
25000/25000 [==============================] - 4s 152us/step - loss: 0.3643 - acc: 0.9492 0s - loss: 0.40
Epoch 3/3
25000/25000 [==============================] - 4s 149us/step - loss: 0.1289 - acc: 0.9948


## Simple Prediction

In [211]:
def prediction(num_to_predict):
    x_new = [np.arange(0, num_to_predict)]
    x_new = sequence.pad_sequences(x_new, dtype='int32', maxlen = 10)
    pred = model.predict(x_new) # Why does it always result in a prediction of 1?
    print(f"Attempting to predict {num_to_predict}: {np.argmax(pred)+1}")
    
# Create a new range of numbers from [0 to X) (not inclusive). Model should predict X
prediction(1) # Makes sense it can't predict 1 because it doesn't know when the sequence starts after the padding
prediction(2)
prediction(3)
prediction(4)
prediction(5)
prediction(9)
prediction(10)


Attempting to predict 1: 2
Attempting to predict 2: 2
Attempting to predict 3: 3
Attempting to predict 4: 4
Attempting to predict 5: 5
Attempting to predict 9: 9
Attempting to predict 10: 10


## Noisy Prediction

In [210]:
# Try a noisy example that isn't just a straight sequence of numbers
x_new = np.arange(0, 7)
x_new[6] = 8
x_new = [x_new]
x_new = sequence.pad_sequences(x_new, dtype='int32', maxlen = 10)
print(x_new)
pred = model.predict(x_new)
# Notice how it knows that the sequence looks more likely to lead up to 7 even though the last digit is 8
print(f"Attempting to predict: {np.argmax(pred)+1}")


[[0 0 0 0 1 2 3 4 5 8]]
Attempting to predict: 7
